In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# 데이터 load
df = pd.read_csv('./DATA/total_dataset_final.csv', engine='python')
df.drop(['Unnamed: 0'], inplace=True, axis=1)

display(df.head())
print()
print('데이터 셋의 갯수 :: ', len(df))
print('데이터 컬럼의 갯수 :: ', len(df.columns))

,SICK_SYM1,MCEX_SICK_SYM,DETAIL_TMSG_SUBJ_CD,SEX,Q_PHX_DX_STK,Q_PHX_DX_HTDZ,Q_PHX_DX_HTN,Q_PHX_DX_DM,Q_PHX_DX_DLD,Q_PHX_DX_PTB,...,Q_HBV_AG,Q_SMK_YN,Q_SMK_PRE_DRT,Q_SMK_PRE_AMT,Q_SMK_NOW_DRT,Q_SMK_NOW_AMT_V09N,Q_DRK_FRQ_V09N,Q_DRK_AMT_V09N,age,TARGET
0,K040,K040,0,2,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,40,55
1,K040,K040,0,2,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,40,55
2,K021,K040,0,2,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,40,55
3,K021,K021,0,2,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,40,55
4,K0538,K0538,0,2,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,40,55



데이터 셋의 갯수 ::  35049
데이터 컬럼의 갯수 ::  33


In [2]:
# LABELING
def data_labeling(dff):
    encoder1 = LabelEncoder()
    encoder1.fit(dff['SICK_SYM1'])
#     np.save('./classes1.npy', encoder1.classes_)
    SICK_SYM1_encoded = encoder1.transform(dff['SICK_SYM1'])

    encoder2 = LabelEncoder()
    encoder2.fit(dff['MCEX_SICK_SYM'])
#     np.save('./classes2.npy', encoder2.classes_)
    MCEX_SICK_SYM_encoded = encoder2.transform(dff['MCEX_SICK_SYM'])

    dff['SICK_SYM1'] = SICK_SYM1_encoded
    dff['MCEX_SICK_SYM'] = MCEX_SICK_SYM_encoded
    
    return encoder1, encoder2, dff

In [3]:
encoder1, encoder2, df_total = data_labeling(df)

X = df.iloc[:, :-1]
y = df.iloc[:, -1:]

# train : test = 7 : 3
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=33)

print('X_train Shape :: ', X_train.shape)
print('y_train Shape :: ', y_train.shape)

X_train Shape ::  (24534, 32)
y_train Shape ::  (24534, 1)


In [4]:
# 학습 진행
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# 예측
y_pred = clf.predict(X_test)

# 정확도 확인
print('정확도 :', metrics.accuracy_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


정확도 : 0.9005230622919639


In [5]:
# 실제 진료과 label, 예측 진료과 label
df_result = pd.DataFrame(y_test['TARGET'].values, columns=['act_label'])
df_result['pred_label'] = y_pred
df_result.head()

,act_label,pred_label
0,1,1
1,1,1
2,1,1
3,50,55
4,1,1


In [6]:
# 결과
total_cnt = len(df_result)
succ_cnt = len(df_result[df_result['act_label'] == df_result['pred_label']])
fail_cnt = len(df_result[df_result['act_label'] != df_result['pred_label']])
acc = succ_cnt / total_cnt * 100
print('전체 테스트 데이터의 수 :: ', total_cnt)
print('성공 데이터의 수 :: ', succ_cnt)
print('실패 데이터의 수 :: ', fail_cnt)
print('정확도 :: %.2f' % acc)

전체 테스트 데이터의 수 ::  10515
성공 데이터의 수 ::  9469
실패 데이터의 수 ::  1046
정확도 :: 90.05
